In [ ]:
import pandas as pd
import numpy as np
import os
from random import randint
from nltk import RegexpTokenizer
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords
import string

In [ ]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir = os.path.join(parent_dir, 'data')
models_dir = os.path.join(parent_dir, 'models')
print('working directory: ', os.getcwd())
print('data directory:    ', data_dir, )
print('models directory:  ', models_dir)

In [ ]:
data = pd.read_pickle(os.path.join(data_dir, 'data_clean_4cols.pickle'))
#data

In [ ]:
candidate_data = pd.DataFrame()
candidate_data = (data[['Partei_ABK', 'from_name']].drop_duplicates('from_name')
                                                   .set_index('from_name'))
# join all messages by the same candidate
candidate_data['messages'] = data.groupby('from_name')['message'].apply(' '.join)

In [ ]:
candidate_data

In [ ]:
n = 50
sample = candidate_data.sample(n=n)

In [ ]:
#tokenizer = RegexpTokenizer(r'\w+')
tokenizer = TweetTokenizer
stopword_set = set(stopwords.words('german'))
#This function does all cleaning of data using two objects above
def nlp_clean(messages):
    new_data = []
    for message in messages:
        message = message.lower()
        message = tokenizer().tokenize(message)
        words = [word for word in message if (word not in stopword_set
                                              and word not in string.punctuation
                                              and not word.startswith('http'))]
        #word_list = tokenizer.tokenize(message)
        #word_list = list(set(dlist).difference(stopword_set))
        new_data.append(words)
    return new_data

In [ ]:
a = 'aaaaa'
not a.startswith('aa')

In [ ]:
documents = nlp_clean(sample['messages'])

In [ ]:
# Choose a random document
i = randint(0, n-1)
i

In [ ]:
d = np.array(np.zeros(n), ndmin=2).T
d[i] = 1
d

In [ ]:
sample['messages'][i]

In [ ]:
documents[i]

In [ ]:
word_list = [word for words in documents for word in words]
vocab = list(set(word_list))

In [ ]:
len(word_list)

In [ ]:
m = len(vocab)
m

In [ ]:
pd.Series(vocab)

In [ ]:
target_word = 'steuern'
idx = vocab.index(target_word)

In [ ]:
t = np.array(np.zeros(m), ndmin=2).T
t[vocab.index(target_word)] = 1
t[idx-5:idx+5]

def get_window(i, window_size=8):
    words = cleaned[i]
    middle = randint(window_size, len(words) - window_size - 1)
    index = np.zeros[len(words)]
    for c in range(middle - window_size, middle + window_size):
        if not c = middle:
            index[c] = 1
    return index

In [ ]:
# use this instead of get_window()
window_size=8
words = documents[i]
middle = randint(window_size, len(words) - window_size - 1)
#window_words = [words[c] for c in range(middle - window_size, middle + window_size)]
#print(window_words)

window_words = []
k = []
for c in range(middle - window_size, middle + window_size):
    window_words.append(words[c])
    kk = (np.array(np.zeros(m), ndmin=2).T)
    kk[vocab.index(words[c])] = 1
    k.append(kk)
k[-1][vocab.index(words[c])-5:vocab.index(words[c])+5]

In [ ]:
window_words

In [ ]:
# p = dimensions of document vectors (no. of features)
p = 100
D = np.random.rand(p, n)
D.shape

In [ ]:
U = np.random.rand(m, p)
U.shape

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)  # axis=1 for row-vector, axis=0 for column-vector

In [ ]:
e = np.array(np.dot(D, d), ndmin=2)
l = np.array(np.dot(U, e), ndmin=2)
k_hat = softmax(l)

In [ ]:
e.shape

In [ ]:
l.shape

In [ ]:
k_hat.shape

In [ ]:
np.sum(k_hat)

In [ ]:
def cross_entropy_loss(k, k_hat):
    return - np.dot(k.T, np.log(k_hat))

In [ ]:
cross_entropy_loss(k[0], k_hat)

In [ ]:
errors_out = k_hat - k[0]
#EI = np.dot(t.T, errors)
#EI
errors_out#.shape

In [ ]:
errors_middle = np.dot(U.T, errors_out)

In [ ]:
alpha = 0.025

In [ ]:
U += - alpha * np.dot(errors_out, e.T)

In [ ]:
D += - alpha * np.dot(errors_middle, d.T)

In [ ]:
window_size=8
alpha_start = 0.025
alpha_end = 0.0001
p = 100  # p = dimensions of document vectors (no. of features)
m = len(vocab)

D = np.random.rand(p, n)
U = np.random.rand(m, p)

In [ ]:
np.array(, ndmin=2).T

In [ ]:
epochs = 20
for e in range(epochs):
    for i in range(n):
        d = np.array(np.zeros(n), ndmin=2).T
        d[i] = 1
        
        e = np.array(np.dot(D, d), ndmin=2)
        l = np.array(np.dot(U, e), ndmin=2)
        k_hat = softmax(l)
        
        #window_size=8
        doc_words = documents[i]
        middle = randint(window_size, len(doc_words) - window_size - 1)
        #window_words = [words[c] for c in range(middle - window_size, middle + window_size)]
        #t[] = np.array(np.zeros(len(vocab)), ndmin=2).T
        #for w in window_words:
        #    t[vocab.index(w)] = 1

        #window_words = []
        #k = []
        errors_out = (np.array(np.zeros(m), ndmin=2).T)
        errors_middle = (np.array(np.zeros(p), ndmin=2).T)
        for c in range(middle - window_size, middle + window_size):
            #window_word = doc_words[c]
            k = (np.array(np.zeros(len(vocab)), ndmin=2).T)
            k[vocab.index(doc_words[c])] = 1
            #k.append(kk)
            errors_out += k_hat - k
            errors_middle += np.dot(U.T, errors_out)
            
            if c == middle:
                print(cross_entropy_loss(k, k_hat))

        errors_out = k_hat - k
        errors_middle = np.dot(U.T, errors_out)
        U += - alpha * np.dot(errors_out, e.T)
        D += - alpha * np.dot(errors_middle, d.T)
        
    

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
tsne = TSNE(n_components=2)

In [ ]:
party_colors = {'AfD': 'xkcd:blue',
                'DIE LINKE': 'xkcd:magenta',
                'GRÜNE': 'xkcd:grass green',
                'CSU': 'xkcd:sky blue',
                'CDU': 'xkcd:black',
                'FDP': 'xkcd:goldenrod',
                'SPD': 'xkcd:red'}
sample['color'] = sample['Partei_ABK'].map(party_colors)

In [ ]:
D_tsne = tsne.fit_transform(D)
plt.figure(num=None, figsize=(10, 8))  # set the figure size
plt.scatter(D_tsne[:, 0], D_tsne[:, 1], c=sample['color'])
plt.show()

In [ ]:
D_tsne = tsne.fit_transform(D)
plt.figure(num=None, figsize=(10, 8))  # set the figure size
plt.scatter(D_tsne[:, 0], D_tsne[:, 1], c=sample['color'])
plt.show()

In [ ]:
D_tsne = tsne.fit_transform(D)
plt.figure(num=None, figsize=(10, 8))  # set the figure size
plt.scatter(D_tsne[:, 0], D_tsne[:, 1], c=sample['color'])
plt.show()

In [ ]:
D_tsne = tsne.fit_transform(D)
plt.figure(num=None, figsize=(10, 8))  # set the figure size
plt.scatter(D_tsne[:, 0], D_tsne[:, 1], c=sample['color'])
plt.show()

In [ ]:
100 * (100 - 5) // 100

In [ ]:
import tensorflow as tf

In [ ]:
# Input data
dataset = tf.placeholder(tf.int32, shape=[80])
labels = tf.placeholder(tf.int32, shape=[80, 1])

# Weights
embeddings = tf.Variable(
        tf.random_uniform([n, p],
                          -1.0, 1.0))
softmax_weights = tf.Variable(
        tf.truncated_normal(
                [m, p],
                stddev=1.0 / np.sqrt(p)))
softmax_biases = tf.Variable(tf.zeros([m]))

# Model
# Look up embeddings for inputs
embed = tf.nn.embedding_lookup(embeddings, dataset)
# Compute the softmax loss, using a sample of the negative
# labels each time
loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(
                softmax_weights, softmax_biases, embed,
                labels, 64, m))

# Optimizer
optimizer = tf.train.AdagradOptimizer(alpha).minimize(loss)

# Test loss
test_loss = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
                tf.matmul(embed, tf.transpose(
                          softmax_weights)) + softmax_biases,
                labels[:, 0]))

# Normalized embeddings (to use cosine similarity later on)
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1,
                              keep_dims=True))
normalized_embeddings = embeddings / norm